In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import random
from collections import Counter

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

pd.set_option('display.max_columns', None)

path = '/Users/aaronng/Downloads/Thesis/'

seed=123
dataset = ''
dataset_suffix = ''
type_suffix = ''

tf.random.set_seed(seed)
np.random.seed(seed)

In [2]:
# Load raw data prepared from data_prep.ipynb
raw_data = pd.read_csv(path + 'raw_data.csv')
raw_data['mode'] = np.where(raw_data['mode'] == 'major', 1, 0)
raw_data.shape

((1617643, 51), (404346, 51), (400599, 51))

In [3]:
# Define 18 unique audio attributes
audio_cols = ['acousticness','beat_strength','bounciness','danceability','dyn_range_mean','energy','flatness','instrumentalness','key','liveness','loudness','mechanism','mode','organism','speechiness','tempo','time_signature','valence']
audio_cols.sort()

# Scale audio attributes
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(raw_data[audio_cols])
data = pd.DataFrame(data, columns=audio_cols)
data = pd.concat([raw_data.drop(audio_cols, axis=1), data], axis=1)
data

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end,track_id,duration,release_year,us_popularity_estimate,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,mode,organism,speechiness,tempo,time_signature,valence
0,22_0000f2fc-143f-41ab-a46e-d24f8de9af52,1,15,t_6af8ddc3-88dd-4107-9d86-ff457a94e095,False,False,False,True,False,False,False,False,0,0,False,7,2018-08-06,True,user_collection,clickrow,trackdone,t_6af8ddc3-88dd-4107-9d86-ff457a94e095,245.933334,2008,99.886278,-0.506540,0.066303,0.308754,0.099178,0.231540,0.138632,-0.248594,-0.032736,0.531082,0.518655,0.594132,0.733411,0.298291,0.567284,0.901674,2.663439e-08,0.818182,0.062346,0.840543,0.662651,1.0,0.458389,0.042157,0.566538,0.8,0.860586
1,22_0000f2fc-143f-41ab-a46e-d24f8de9af52,2,15,t_6017a19e-c80b-44aa-8082-027d7a239cf7,False,False,True,False,False,True,False,False,0,0,False,7,2018-08-06,True,user_collection,trackdone,fwdbtn,t_6017a19e-c80b-44aa-8082-027d7a239cf7,232.759995,2008,99.595876,-0.634187,0.101961,0.317427,0.199928,0.197700,0.099790,-0.152495,0.050981,0.264632,0.673539,0.771968,0.878907,0.387477,0.518875,0.920068,3.116417e-07,0.636364,0.086191,0.827114,0.824480,1.0,0.231372,0.048352,0.533812,0.8,0.863900
2,22_0000f2fc-143f-41ab-a46e-d24f8de9af52,3,15,t_fa5ab487-ab49-48ef-adb5-f987679bd4c4,False,True,True,False,False,True,False,False,0,0,False,7,2018-08-06,True,user_collection,fwdbtn,fwdbtn,t_fa5ab487-ab49-48ef-adb5-f987679bd4c4,256.266663,2008,99.740290,-0.648128,0.079560,0.394614,0.251039,0.112599,0.099564,-0.121302,-0.013774,0.139713,0.507004,0.623774,0.756456,0.321718,0.613629,0.906907,3.085022e-10,0.000000,0.338412,0.847443,0.694073,1.0,0.245588,0.049910,0.582907,0.8,0.682999
3,22_0000f2fc-143f-41ab-a46e-d24f8de9af52,4,15,t_c4faeeae-e295-4752-b015-b45257803a45,True,True,True,False,False,True,False,False,0,0,False,7,2018-08-06,True,user_collection,fwdbtn,fwdbtn,t_c4faeeae-e295-4752-b015-b45257803a45,222.679993,2008,99.693158,-0.013097,0.287468,0.378081,-0.294970,-0.013921,0.177360,-0.211256,-0.519585,0.879086,0.375120,0.456012,0.557308,0.254669,0.254053,0.935116,1.486917e-08,0.818182,0.110785,0.798199,0.315166,1.0,0.812188,0.031424,0.549401,0.8,0.297879
4,22_0000f2fc-143f-41ab-a46e-d24f8de9af52,5,15,t_14de5c5c-6aa2-41cd-81ad-e4768f7d7691,False,False,False,True,False,True,False,False,0,0,False,7,2018-08-06,True,user_collection,fwdbtn,trackdone,t_14de5c5c-6aa2-41cd-81ad-e4768f7d7691,231.573334,2008,99.701243,-0.599022,0.029001,0.390504,0.217310,0.157821,0.132032,-0.202796,-0.014954,0.238445,0.515255,0.564583,0.667892,0.280536,0.699289,0.901278,1.039508e-08,0.909091,0.078327,0.839847,0.832075,1.0,0.212519,0.043949,0.656963,0.8,0.958724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1617638,9_fffd3e23-c58f-4c9e-b593-ede93b81c45f,16,20,t_aae74c4e-7ac0-4ffc-8560-88303a918c3f,False,False,False,True,False,True,False,False,0,0,False,19,2018-07-24,True,editorial_playlist,backbtn,trackdone,t_aae74c4e-7ac0-4ffc-8560-88303a918c3f,417.920013,2018,99.999883,-0.821723,0.397023,0.409666,0.332433,-0.007174,0.078549,0.266564,0.242550,0.544404,0.741114,0.833177,0.906482,0.428306,0.674989,0.913367,1.280319e-05,1.000000,0.059543,0.874950,0.732113,0.0,0.441879,0.220165,0.396321,0.8,0.442089
1617639,9_fffd3e23-c58f-4c9e-b593-ede93b81c45f,17,20,t_f55e6fef-b23e-4b18-b7a5-2462bfbee4fd,False,False,F

In [5]:
# Calculate average standard deviation across sessions for each feature to be used in the prior
avg_stds = data[['session_id'] + audio_cols].groupby('session_id').std().mean()
avg_stds

acousticness        0.197037
beat_strength       0.115689
bounciness          0.126534
danceability        0.120398
dyn_range_mean      0.058750
energy              0.145211
flatness            0.024500
instrumentalness    0.049355
key                 0.323244
liveness            0.125188
loudness            0.034569
mechanism           0.172836
mode                0.452812
organism            0.158220
speechiness         0.097477
tempo               0.114228
time_signature      0.038231
valence             0.191649
dtype: float64

## State Extraction Model using HMM

In [6]:
NUM_EXAMPLES = 10000 # Note: A total of 50,000 sesssions were analysed in original paper
NUM_STATES = 3
MAX_SESSION_LENGTH = 20

In [7]:
initial_state_logits = np.zeros([NUM_STATES]) # uniform distribution

daily_change_prob = 1 - 1/NUM_STATES
transition_probs = daily_change_prob / (NUM_STATES-1) * np.ones([NUM_STATES, NUM_STATES])
np.fill_diagonal(transition_probs, 1-daily_change_prob)

print("Initial state logits:\n{}".format(initial_state_logits))
print("Transition matrix:\n{}".format(transition_probs))

Initial state logits:
[0. 0. 0.]
Transition matrix:
[[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]]


In [8]:
# Take a sampled subset of sessions

sess_ids = pd.Series(data['session_id'].unique()).sample(NUM_EXAMPLES, random_state=seed).sort_values()
sampled_data = data[data['session_id'].isin(sess_ids)].reset_index(drop=True)

In [9]:
# Prepare tensor of audio attributes X no. of sessions X session length
# Pad sessions which are shorter than 20 with zeros
# Also, calculate mean & std for each session

observed_values = [sampled_data[['session_id', col]].groupby('session_id') for col in audio_cols]
observed_values = observed_means_stds = [[feat.get_group(x).iloc[:,1].values for x in feat.groups] for feat in observed_values]

observed_values = np.array([np.stack([np.pad(sess, (0,MAX_SESSION_LENGTH-len(sess)), 'edge') for sess in feat]) for feat in observed_values])
observed_means_stds = np.array([np.stack([[np.mean(sess), np.std(sess)] for sess in feat]) for feat in observed_means_stds])
    
observed_values = np.array(observed_values).astype('float32')
observed_means_stds = np.array(observed_means_stds).astype('float32')

observed_values.shape, observed_means_stds.shape

((18, 10000, 20), (18, 10000, 2))

In [10]:
observed_values = observed_values.reshape(len(audio_cols) * NUM_EXAMPLES,MAX_SESSION_LENGTH)
observed_means_stds = observed_means_stds.reshape(len(audio_cols) * NUM_EXAMPLES,2)

trainable_means = tf.Variable(
    (observed_means_stds[:,0].reshape(-1,1) * np.ones(NUM_STATES) + tf.random.normal([1, NUM_STATES], seed=seed)),
    name='means')

print(observed_values.shape, observed_means_stds.shape, trainable_means.shape)

(180000, 20) (180000, 2) (180000, 3)


In [11]:
# Define std for each feature in each session to be the avg std of that feature across all sessions

stds = []
for i, col in enumerate(audio_cols):
    if stds == []:
        stds = avg_stds[col] * np.ones([NUM_EXAMPLES, NUM_STATES]).astype('float32')
    else:
        stds = np.vstack([stds, avg_stds[col] * np.ones([NUM_EXAMPLES, NUM_STATES]).astype('float32')])
stds = stds.astype('float32')
stds.shape

/Users/aaronng/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


(180000, 3)

In [17]:
# Define a HMM with a categorical distribution for initial and transition distribution,
# and normal distribution for observation distribution

hmm = tfd.HiddenMarkovModel(
      initial_distribution=tfd.Categorical(logits=initial_state_logits),
      transition_distribution=tfd.Categorical(probs=transition_probs),
      observation_distribution=tfd.Normal(trainable_means, stds),
      num_steps=MAX_SESSION_LENGTH)

In [18]:
## Apply each session's prior to all states, summing them up, before adding to log_prob of observed values

def log_prob():
  prior = tfd.Normal(observed_means_stds[:,0], stds[:,0].astype('float32'))
  return prior.log_prob(trainable_means[:,0]) \
+ prior.log_prob(trainable_means[:,1]) \
+ prior.log_prob(trainable_means[:,2]) \
+ hmm.log_prob(observed_values)

In [19]:
# Train HMM using Adam optimizer

import time
start_time = time.time()

@tf.function(autograph=False)
def train_op():
  with tf.GradientTape() as tape:
    neg_log_prob = -log_prob()
  grads = tape.gradient(neg_log_prob, [trainable_means])[0]
  optimizer.apply_gradients([(grads, trainable_means)])
  return neg_log_prob, trainable_means

optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
for step in range(201):
  loss, means = [t.numpy() for t in train_op()]
  if step % 20 == 0:
    print("step {}: loss {}".format(step, -loss))

print('Time taken: {}'.format(time.time() - start_time))

step 0: loss [-2.0409622  8.716383   9.560743  ...  4.182269   7.237394   8.530246 ]
step 20: loss [-1.6317892  9.305063  10.14873   ...  4.7720857  7.883154   9.158319 ]
step 40: loss [-1.6294601  9.297364  10.141     ...  4.7710953  7.876329   9.150461 ]
step 60: loss [-1.6233239  9.307854  10.151488  ...  4.7815127  7.8873873  9.161541 ]
step 80: loss [-1.622673   9.309084  10.1527195 ...  4.782853   7.8887086  9.162846 ]
step 100: loss [-1.6225953  9.309164  10.152799  ...  4.7829247  7.88879    9.16293  ]
step 120: loss [-1.6225791  9.309172  10.152805  ...  4.7829323  7.8887978  9.162937 ]
step 140: loss [-1.622576   9.3091755 10.152809  ...  4.7829356  7.8888006  9.16294  ]
step 160: loss [-1.6225764  9.309175  10.152809  ...  4.782935   7.8888006  9.16294  ]
step 180: loss [-1.6225755  9.3091755 10.152809  ...  4.782936   7.888802   9.162941 ]
step 200: loss [-1.6225774  9.3091755 10.152809  ...  4.782936   7.8888016  9.162941 ]
Time taken: 231.26942610740662


In [20]:
# Calculate posterior marginals and find most probable state for each timestep
posterior_probs = hmm.posterior_marginals(observed_values).probs_parameter().numpy()
most_probable_states = np.argmax(posterior_probs, axis=-1)

In [21]:
# No. of states per audio attribute before merging similar states

num_states_features = pd.DataFrame(pd.Series(np.arange(1,11), name='index'))

for i, col in enumerate(audio_cols):
    num_states = pd.DataFrame(most_probable_states[i * NUM_EXAMPLES:(i+1)* NUM_EXAMPLES]) \
                .nunique(axis=1).value_counts().sort_index().reset_index().rename(columns={0:col})
    num_states_features = num_states_features.merge(num_states, on='index', how='left')

num_states_features.set_index('index')

,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,mode,organism,speechiness,tempo,time_signature,valence
index,,,,,,,,,,,,,,,,,,
1,532.0,618.0,717.0,626.0,156.0,1051.0,5.0,3738.0,246.0,661.0,27.0,665.0,603.0,1035.0,750.0,592.0,4862.0,531.0
2,5516.0,5322.0,5436.0,5296.0,4605.0,5536.0,5357.0,3390.0,6471.0,5926.0,4556.0,5581.0,6889.0,5565.0,5201.0,5693.0,2783.0,5858.0
3,3952.0,4060.0,3847.0,4078.0,5239.0,3413.0,4638.0,2872.0,3283.0,3413.0,5417.0,3754.0,2508.0,3400.0,4049.0,3715.0,2355.0,3611.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
truncate = np.vectorize(lambda n: int(n * 100) / 100)

def merge_states(states, means):
    # Round down to nearest 0.01 and group similar states
    means = truncate(means) 
    grouped_states = [np.where(means == i)[0].tolist() for i in np.unique(means)]
    
    # Map each state to first state in group
    map_states = {}
    for group in grouped_states:
        for i in group:
            map_states[i] = group[0]

    fn = np.vectorize(lambda x: map_states[x])
    merged_states = fn(states)
    
    ## Replace states if count < 3
    # Count states
    state_counts = Counter(merged_states)
    valid_states = [state for state in state_counts.keys() if state_counts[state] >= 3]
    invalid_states = list(set(state_counts.keys()) - set(valid_states))
    
    # Simply replace if there's only 1 valid state
    if len(valid_states) == 1:
        merged_states = [valid_states[0] for s in merged_states]
    elif len(invalid_states) == 0:
        merged_states = merged_states
    else:
        for inv_state in invalid_states:
            # Find nearest valid state and merge with it
            dist_to_valid_states = [np.abs(means[state] - means[inv_state]) for state in valid_states]
            state_to_merge_with = valid_states[np.argmin(dist_to_valid_states)]
            merged_states = [state_to_merge_with if s == inv_state else s for s in merged_states]
    return merged_states

In [23]:
# Merge similar states
most_probable_states_merged = [merge_states(s,m) for s,m in zip(most_probable_states,means)]

In [24]:
# No. of states per audio attribute after merging similar states

num_states_features = pd.DataFrame(pd.Series(np.arange(1,11), name='index'))

for i, col in enumerate(audio_cols):
    num_states = pd.DataFrame(most_probable_states_merged[i * NUM_EXAMPLES:(i+1)* NUM_EXAMPLES]) \
                .nunique(axis=1).value_counts().sort_index().reset_index().rename(columns={0:col})
    num_states_features = num_states_features.merge(num_states, on='index', how='left')

num_states_features = num_states_features.rename(columns={'index':'num_states'})
num_states_features = num_states_features.set_index('num_states')
num_states_features

,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,mode,organism,speechiness,tempo,time_signature,valence
num_states,,,,,,,,,,,,,,,,,,
1,6271.0,6868.0,6843.0,6946.0,6858.0,6911.0,7247.0,9102.0,7840.0,7201.0,7233.0,6623.0,6080.0,6704.0,6237.0,7212.0,8731.0,7159.0
2,3710.0,2990.0,3050.0,2957.0,2976.0,3004.0,2591.0,733.0,2158.0,2778.0,2608.0,3347.0,3920.0,3248.0,3629.0,2753.0,1226.0,2806.0
3,19.0,142.0,107.0,97.0,166.0,85.0,162.0,165.0,2.0,21.0,159.0,30.0,NaN,48.0,134.0,35.0,43.0,35.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Determine audio states per session

In [25]:
# Get skip_2 column for all sessions, padded up to MAX_SESSION_LENGTH for sessions with shorter length
observed_skips = sampled_data[['session_id', 'skip_2']].groupby('session_id')
observed_skips = [observed_skips.get_group(x).iloc[:,1].values for x in observed_skips.groups]
observed_skips = np.stack([np.pad(sess, (0,MAX_SESSION_LENGTH-len(sess)), 'edge') for sess in observed_skips])
observed_skips = observed_skips.astype(int)
observed_skips.shape

(10000, 20)

In [26]:
# Get session length for all sessions
session_lengths = sampled_data[['session_id','session_length']].groupby('session_id').mean()['session_length'].values
session_lengths.shape

(10000,)

In [ ]:
# Replace states by their respective avg_skips and use that as a score to rank tracks later
sess_feat_scores = []
sess_feat_states = []

for i, (sess_states_padded,sess_means_padded) in enumerate(zip(most_probable_states_merged,means)):
    idx = i % NUM_EXAMPLES
    sess_states_i = sess_states_padded[:session_lengths[idx]]

    session = pd.DataFrame(sess_states_i, columns=['states'])
    session['skip_2'] = observed_skips[idx][:session_lengths[idx]]
    
    states_avg_skips_dict = session.groupby('states').agg(lambda x: np.sum(x) / len(x)).to_dict('index')
    sess_feat_score = [states_avg_skips_dict[state]['skip_2'] for state in sess_states_i]

    sess_feat_scores.append(sess_feat_score)
    sess_feat_states.append(sess_states_i)
    
    if i%1000 == 0:
        print('Processed {} sequences'.format(i))

In [28]:
# Score (avg skips) for each audio attribute of each track of each session 
sess_scores = np.array([np.array(x) for x in sess_feat_scores]).reshape(len(audio_cols), NUM_EXAMPLES)
sess_scores = [np.hstack(feat) for feat in sess_scores]
sess_scores = pd.DataFrame(np.array(sess_scores).transpose(), columns=audio_cols)

sess_scores[['session_id','session_position','skip_2']] = sampled_data[['session_id','session_position','skip_2']]
sess_scores = sess_scores[['session_id','session_position','skip_2'] + audio_cols]
sess_scores['skip_2'] = sess_scores['skip_2'].astype(int)
sess_scores['relevance'] = sess_scores['skip_2'] ^ 1
sess_scores.drop('skip_2',axis=1,inplace=True)

sess_scores

,session_id,session_position,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,mode,organism,speechiness,tempo,time_signature,valence,relevance
0,36_0007583d-ca87-4edc-8282-987bd32c95be,1,0.076923,0.2,0.2,0.2,0.2,0.133333,0.2,0.2,0.2,0.2,0.125,0.3,0.2,0.133333,0.2,0.181818,0.2,0.4,0
1,36_0007583d-ca87-4edc-8282-987bd32c95be,2,0.428571,0.2,0.2,0.2,0.2,0.400000,0.2,0.2,0.2,0.2,0.500,0.3,0.2,0.400000,0.2,0.181818,0.2,0.4,0
2,36_0007583d-ca87-4edc-8282-987bd32c95be,3,0.428571,0.2,0.2,0.2,0.2,0.133333,0.2,0.2,0.2,0.2,0.125,0.1,0.2,0.133333,0.2,0.222222,0.2,0.4,0
3,36_0007583d-ca87-4edc-8282-987bd32c95be,4,0.428571,0.2,0.2,0.2,0.2,0.400000,0.2,0.2,0.2,0.2,0.500,0.3,0.2,0.400000,0.2,0.222222,0.2,0.4,0
4,36_0007583d-ca87-4edc-8282-987bd32c95be,5,0.428571,0.2,0.2,0.2,0.2,0.133333,0.2,0.2,0.2,0.2,0.125,0.1,0.2,0.133333,0.2,0.222222,0.2,0.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160512,36_ffee4c66-6f90-4300-a355-bd955611c49c,11,0.400000,0.4,0.4,0.4,0.4,0.400000,0.4,0.4,0.4,0.4,0.400,0.4,0.4,0.400000,0.4,0.400000,0.4,0.4,1
160513,36_ffee4c66-6f90-4300-a355-bd955611c49c,12,0.400000,0.4,0.4,0.4,0.4,0.400000,0.4,0.4,0.4,0.4,0.400,0.4,0.4,0.400000,0.4,0.400000,0.4,0.4,1
160514,36_ffee4c66-6f90-4300-a355-bd955611c49c,13,0.400000,0.4,0.4,0.4,0.4,0.400000,0.4,0.4,0.4,0.4,0.400,0.4,0.4,0.400000,0.4,0.400000,0.4,0.4,1
160515,36_ffee4c66-6f90-4300-a355-bd955611c49c,14,0.400000,0.4,0.4,0.4,0.4,0.400000,0.4,0.4,0.4,0.4,0.400,0.4,0.4,0.400000,0.4,0.400000,0.4,0.4,1


In [29]:
# State for each audio attribute of each track of each session 
sess_states = np.array([np.array(x) for x in sess_feat_states]).reshape(len(audio_cols), NUM_EXAMPLES)
sess_states = [np.hstack(feat) for feat in sess_states]
sess_states = pd.DataFrame(np.array(sess_states).transpose(), columns=audio_cols)

sess_states[['session_id','session_position','skip_2']] = sampled_data[['session_id','session_position','skip_2']]
sess_states = sess_states[['session_id','session_position','skip_2'] + audio_cols]
sess_states['skip_2'] = sess_states['skip_2'].astype(int)
sess_states['relevance'] = sess_states['skip_2'] ^ 1
sess_states.drop('skip_2',axis=1,inplace=True)

sess_states

,session_id,session_position,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,mode,organism,speechiness,tempo,time_signature,valence,relevance
0,36_0007583d-ca87-4edc-8282-987bd32c95be,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,36_0007583d-ca87-4edc-8282-987bd32c95be,2,2,0,0,0,0,1,0,0,0,0,2,0,0,1,0,0,1,0,0
2,36_0007583d-ca87-4edc-8282-987bd32c95be,3,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,1,0,0
3,36_0007583d-ca87-4edc-8282-987bd32c95be,4,2,0,0,0,0,1,0,0,0,0,2,0,0,1,0,2,1,0,0
4,36_0007583d-ca87-4edc-8282-987bd32c95be,5,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160512,36_ffee4c66-6f90-4300-a355-bd955611c49c,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
160513,36_ffee4c66-6f90-4300-a355-bd955611c49c,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
160514,36_ffee4c66-6f90-4300-a355-bd955611c49c,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
160515,36_ffee4c66-6f90-4300-a355-bd955611c49c,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [ ]:
# Store audio states to file
sess_states.to_csv(path + 'states-{}{}{}.csv'.format(dataset, dataset_suffix, type_suffix), index=False)

## Determine top audio attributes per session

In [125]:
K=10

metric_cols = ['NDCG@{}_{}'.format(K,col) for col in audio_cols]
metric_cols = ['session_id','NDCG@{}'.format(K)] + metric_cols

metric_cols_all = ['{}@{}_{}'.format(metric,K,col) for col in audio_cols for metric in ['NDCG','AP','P','RR']]
metric_cols_all = ['session_id','NDCG@{}'.format(K),'AP@{}'.format(K),'P@{}'.format(K),'RR@{}'.format(K)] + metric_cols_all

In [129]:
from sklearn.metrics import f1_score

# Metric definitions
# https://gist.github.com/bwhite/3726239/
def dcg_score(y_score, k=999):
    y = y_score[:k]
    score = 0
    
    for i in range(len(y)):
        score += (2**y[i] - 1) / np.log2(i+2)

    return np.sum(score)

def ndcg_score(y_score, k=999):
    y = y_score[:k]
    
    idcg = dcg_score(sorted(y_score, reverse=True), k)
    return dcg_score(y_score, k) / idcg if idcg > 0 else 0

def precision_score(y_score, k=999):
    cumulative_score = 0
    num_pos = 0
    y = y_score[:k]
    
    return np.sum(y) / len(y)

def average_precision_score(y_score, k=999):
    cumulative_score = 0
    num_pos = 0
    y = y_score[:k]
    
    for i in range(len(y)):
        if y[i] == 1:
            num_pos += 1
            cumulative_score += num_pos / (i+1)
    
    return cumulative_score / num_pos if num_pos > 0 else 0

def reciprocal_rank(y_score, k=999):
    y = y_score[:k]
    
    try:
        return 1/(y.index(1)+1)
    except ValueError:
        return 0

# Method to re-rank session and calculate ranking metrics
def calc_metrics(results, K=10, feature_cols=audio_cols, metric_cols=metric_cols, all_metrics=False):
    metrics = []
    count=0

    for session_id in results['session_id'].unique():
        session_results = results[results['session_id'] == session_id]
        y = list(session_results.reset_index(drop=True)['relevance'])
        
        ndcg_K = ndcg_score(y, K)
        session_metrics = [session_id, ndcg_K]
        
        if all_metrics == True:
            ap_K = average_precision_score(y, K)
            p_K = precision_score(y, K)
            rr_K = reciprocal_rank(y, K)
            session_metrics = [session_id, ndcg_K, ap_K, p_K, rr_K]
        
        for col in feature_cols:
            session_results_col = session_results[[col, 'session_position', 'relevance']]
            
            # Return current score if there's only 1 state
            if session_results_col[col].nunique() == 1:
                ndcg_K_i = ndcg_K
                
                if all_metrics == True:
                    ap_K_i = ap_K
                    p_K_i = p_K
                    rr_K_i = rr_K
                    
            else:
                # Rank states with low avg_skips higher and tie-break by session position
                session_results_col.sort_values([col, 'session_position'], inplace=True)
                ndcg_K_i = ndcg_score(list(session_results_col['relevance']), K)
                
                if all_metrics == True:
                    ap_K_i = average_precision_score(list(session_results_col['relevance']), K)
                    p_K_i = precision_score(list(session_results_col['relevance']), K)
                    rr_K_i = reciprocal_rank(list(session_results_col['relevance']), K)
            
            if all_metrics == False:
                session_metrics += [ndcg_K_i]
            else:
                session_metrics += [ndcg_K_i, ap_K_i, p_K_i, rr_K_i]
        
        metrics.append(session_metrics)

        count+=1
        if count%100 == 0:
            print('Processed {} sessions'.format(count))
            
    return pd.DataFrame(metrics, columns=metric_cols)

In [ ]:
metrics = calc_metrics(sess_scores)
metrics

In [33]:
# Calculate the delta between the ranking score before and after re-ranking.
# Positive means that the audio attribute has helped to improve the ranking of the session
ndcg_orig = metrics[['session_id','NDCG@10']].set_index('session_id')
ndcg_above_orig = metrics.set_index('session_id').drop('NDCG@10', axis=1)
ndcg_above_orig = ndcg_above_orig.sub(ndcg_orig['NDCG@10'],axis=0)
ndcg_above_orig.columns = audio_cols
ndcg_above_orig

,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,mode,organism,speechiness,tempo,time_signature,valence
session_id,,,,,,,,,,,,,,,,,,
36_0007583d-ca87-4edc-8282-987bd32c95be,0.343697,-0.436212,-0.436212,-0.436212,-0.436212,0.204834,-0.436212,-0.436212,-0.436212,-0.436212,0.204834,0.343697,-0.436212,0.204834,-0.436212,0.204834,-0.436212,0.563788
36_00112b15-232f-49ab-8011-0067ce480dd1,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,0.167702,-0.629552,-0.629552,-0.018714,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552
36_00128b9b-285e-47a2-bb2b-2127fde65f2a,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.071233,-0.390380,-0.390380,-0.390380
36_001340c4-d263-4c7f-bda9-f2a4261774e6,-0.374938,-0.758587,-0.435000,-0.435000,-0.758587,-0.758587,-0.758587,-0.758587,-0.758587,-0.758587,0.034279,-0.435000,-0.758587,-0.435000,-0.289308,-0.758587,-0.758587,-0.758587
36_0013a0d9-005e-475d-b698-18df4fcb3d41,0.110046,0.210670,0.113223,0.339125,0.232095,0.175544,0.248908,-0.531000,-0.531000,0.110046,-0.531000,0.269694,-0.531000,0.405379,0.185288,-0.531000,-0.531000,0.110046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36_ffc522eb-ecb8-4d6e-acbd-4afc4b28c099,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,0.015712,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,0.018889,-0.914857
36_ffcd0f76-e154-45fe-b561-c38f86729852,0.194709,0.128455,0.128455,-0.741670,-0.741670,-0.741670,-0.741670,-0.741670,-0.741670,-0.741670,-0.741670,0.053496,0.119467,0.078398,-0.741670,-0.741670,-0.741670,0.258330
36_ffd6d90d-f140-416c-80f1-30bef230e5ee,0.102097,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.286116,-0.651186,0.022772,-0.651186,0.028832,-0.651186,0.135825


In [34]:
# Determine the top audio attributes for each session, sorted from best to worst
top_cols = ['top{}'.format(i) for i in range(1,len(audio_cols)+1)]

def replace_0_with_NONE(x):
    tmp = x.reset_index()
    tmp = tmp.sort_values([tmp.columns[1], tmp.columns[0]], ascending=[False, True])
    # Ignore features which score worse than current. Features that perform same as current is still indicative
    tmp.loc[tmp.iloc[:,1] < 0, 'index'] = 'NONE'
    return tmp

topN_features_ndcg = ndcg_above_orig.apply(
    lambda x: pd.Series(replace_0_with_NONE(x)['index'].values, 
    index=top_cols), 
    axis=1
).reset_index()

topN_features_ndcg

,session_id,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18
0,36_0007583d-ca87-4edc-8282-987bd32c95be,valence,acousticness,mechanism,energy,loudness,organism,tempo,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
1,36_00112b15-232f-49ab-8011-0067ce480dd1,liveness,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
2,36_00128b9b-285e-47a2-bb2b-2127fde65f2a,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
3,36_001340c4-d263-4c7f-bda9-f2a4261774e6,loudness,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
4,36_0013a0d9-005e-475d-b698-18df4fcb3d41,organism,danceability,mechanism,flatness,dyn_range_mean,beat_strength,speechiness,energy,bounciness,acousticness,liveness,valence,NONE,NONE,NONE,NONE,NONE,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,36_ffc522eb-ecb8-4d6e-acbd-4afc4b28c099,time_signature,liveness,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
9996,36_ffcd0f76-e154-45fe-b561-c38f86729852,valence,acousticness,beat_strength,bounciness,mode,organism,mechanism,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
9997,36_ffd6d90d-f140-416c-80f1-30bef230e5ee,valence,acousticness,tempo,organism,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
9998,36_ffe5f35f-21c4-47c7-9675-3c0f4c6c8f35,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE


In [35]:
# NDCG@10 delta of each session, sorted from best to worst
topN_features_ndcg_values = ndcg_above_orig.apply(
    lambda x: pd.Series(replace_0_with_NONE(x).iloc[:,1].values, 
    index=top_cols), 
    axis=1
).reset_index()

topN_features_ndcg_values

,session_id,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18
0,36_0007583d-ca87-4edc-8282-987bd32c95be,0.563788,0.343697,0.343697,0.204834,0.204834,0.204834,0.204834,-0.436212,-0.436212,-0.436212,-0.436212,-0.436212,-0.436212,-0.436212,-0.436212,-0.436212,-0.436212,-0.436212
1,36_00112b15-232f-49ab-8011-0067ce480dd1,0.167702,-0.018714,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552,-0.629552
2,36_00128b9b-285e-47a2-bb2b-2127fde65f2a,-0.071233,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380,-0.390380
3,36_001340c4-d263-4c7f-bda9-f2a4261774e6,0.034279,-0.289308,-0.374938,-0.435000,-0.435000,-0.435000,-0.435000,-0.758587,-0.758587,-0.758587,-0.758587,-0.758587,-0.758587,-0.758587,-0.758587,-0.758587,-0.758587,-0.758587
4,36_0013a0d9-005e-475d-b698-18df4fcb3d41,0.405379,0.339125,0.269694,0.248908,0.232095,0.210670,0.185288,0.175544,0.113223,0.110046,0.110046,0.110046,-0.531000,-0.531000,-0.531000,-0.531000,-0.531000,-0.531000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,36_ffc522eb-ecb8-4d6e-acbd-4afc4b28c099,0.018889,0.015712,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857,-0.914857
9996,36_ffcd0f76-e154-45fe-b561-c38f86729852,0.258330,0.194709,0.128455,0.128455,0.119467,0.078398,0.053496,-0.741670,-0.741670,-0.741670,-0.741670,-0.741670,-0.741670,-0.741670,-0.741670,-0.741670,-0.741670,-0.741670
9997,36_ffd6d90d-f140-416c-80f1-30bef230e5ee,0.135825,0.102097,0.028832,0.022772,-0.286116,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186,-0.651186
9998,36_ffe5f35f-21c4-47c7-9675-3c0f4c6c8f35,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766,-0.820766


In [36]:
# Value count of audio attributes in the topN features
topN_features_ndcg_stats = topN_features_ndcg.iloc[:,1:].apply(pd.Series.value_counts)
topN_features_ndcg_stats

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18
NONE,1173,2680,4068,5347,6440,7332,8065,8651,9127,9448,9664,9823,9919,9969.0,9988.0,9998.0,10000.0,10000.0
acousticness,771,549,403,315,229,184,128,104,42,49,20,9,4,1.0,NaN,NaN,NaN,NaN
beat_strength,598,447,336,308,220,148,112,63,43,18,12,3,1,NaN,NaN,NaN,NaN,NaN
bounciness,324,567,414,311,236,182,122,83,49,23,12,10,5,NaN,NaN,NaN,NaN,NaN
danceability,419,361,387,292,255,192,125,91,50,34,19,11,4,3.0,NaN,NaN,NaN,NaN
dyn_range_mean,397,410,386,355,228,198,128,89,50,32,12,10,3,1.0,NaN,NaN,NaN,NaN
energy,526,443,347,272,200,162,106,103,48,33,16,5,5,4.0,NaN,NaN,NaN,NaN
flatness,489,390,301,259,170,143,102,65,47,34,19,15,5,NaN,2.0,NaN,NaN,NaN
instrumentalness,198,100,106,74,70,46,45,35,25,11,13,7,2,5.0,2.0,NaN,NaN,NaN
key,511,337,229,149,108,85,52,32,18,16,7,5,2,NaN,1.0,NaN,NaN,NaN


In [37]:
sess_ids_top12 = list(topN_features_ndcg_values[(topN_features_ndcg_values['top1'] == topN_features_ndcg_values['top2']) & (topN_features_ndcg_values['top3'] != topN_features_ndcg_values['top2'])]['session_id'])
sess_ids_top123 = list(topN_features_ndcg_values[(topN_features_ndcg_values['top1'] == topN_features_ndcg_values['top2']) & (topN_features_ndcg_values['top3'] == topN_features_ndcg_values['top2'])]['session_id'])
sess_ids_top1 = list(topN_features_ndcg_values[topN_features_ndcg_values['top1'] != topN_features_ndcg_values['top2']]['session_id'])

print('num sessions top1:', len(sess_ids_top1)/NUM_EXAMPLES)
print('num sessions top1==top2:', len(sess_ids_top12)/NUM_EXAMPLES)
print('num sessions top1==top2==top3:', len(sess_ids_top123)/NUM_EXAMPLES)

num sessions top1: 0.7822
num sessions top1==top2: 0.108
num sessions top1==top2==top3: 0.1098


In [ ]:
# Store top audio attributes to file
topN_features_ndcg.to_csv(path + 'topN-features-{}{}{}.csv'.format(dataset, dataset_suffix, type_suffix), index=False)